In [1]:
import numpy as np
import tensorflow as tf
import logging

In [2]:
from transformer_modules import *
from utils import *

In [3]:
np.random.seed(20)

In [4]:
N_ = 1000
n_seq = 15
max_val = 10
inp_data = np.random.randint(low=0, high=max_val, size = (N_,n_seq))
out_data = np.concatenate([np.full([N_, 1], np.max(inp_data)+1), inp_data], axis = 1)
data = (inp_data, out_data)

batch_size=10
batch_iter = gen_batch(data, batch_size=batch_size)

In [5]:
#helper function for placeholder feeding
def get_labels(dec_inp_batch):
    lbls = np.zeros([*dec_inp_batch.shape, max_val+1])
    for i in range(lbls.shape[0]):
        for j in range(lbls.shape[1]):
            lbls[i,j,dec_inp_batch[i,j]] = 1
    return lbls

In [6]:
#placeholders
enc_inp_seq = tf.placeholder(shape=(batch_size, n_seq), dtype=tf.int32, name='enc_inp_plhd') 
dec_inp_seq = tf.placeholder(shape=(batch_size, n_seq), dtype=tf.int32, name='dec_inp_plhd') #out_embs(next_batch[1][:,:-1])
out_lbls = tf.placeholder(shape=(batch_size, n_seq, max_val+1), dtype=tf.int32, name='out_plhd') #

In [7]:
d_model = 50

inp_embs = WordEmbeddings(d_model=d_model, d_vocab=max_val, name='inp_embs')
out_embs = WordEmbeddings(d_model=d_model, d_vocab=max_val+1, name='out_embs')

tsf_enc_inp = inp_embs(enc_inp_seq) #inp_embs(next_batch[0])
tsf_dec_inp = out_embs(dec_inp_seq) #out_embs(next_batch[1][:,:-1])

In [8]:
mask = subsequent_mask(n_seq)
mask = tf.expand_dims(tf.constant(mask), axis=0)

In [9]:
logger.setLevel(5)

In [10]:
# mask=None

In [11]:
model = TransformerModel(d_inp_vocab=max_val, d_out_vocab=max_val+1, d_model=d_model, n_blocks=1, n_heads=5, d_ff=100, dropout=0)

out = model(tsf_enc_inp, tsf_dec_inp, tgt_mask=mask)

In [12]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=out_lbls, logits=out, dim=-1))
global_step = tf.Variable(0, trainable=False)
train_step = tf.train.AdamOptimizer(learning_rate=0.003, name = 'adam').minimize(loss, global_step=global_step)

In [13]:
correct_preds = tf.equal(tf.argmax(out, axis=-1), tf.argmax(out_lbls, axis=-1))
accuracy = tf.reduce_mean(tf.cast(correct_preds, tf.float32))

In [14]:
config = tf.ConfigProto(device_count = {'GPU': 0})
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())

In [15]:
for epoch in range(4):
    batch_iter = gen_batch(data, batch_size=batch_size)
    for i, next_batch in enumerate(batch_iter):
        #next_batch = batch_iter.__next__()
        if len(next_batch[0]) != 0:
            l, _, acc = sess.run([loss, train_step, accuracy], feed_dict={enc_inp_seq: next_batch[0], dec_inp_seq:next_batch[1][:,:-1], out_lbls: get_labels(next_batch[1][:,1:])})
            if i % 50 == 0:
                print (i,l, acc)

0 4.3190265 0.08
50 2.1745164 0.18666667
0 1.6591218 0.41333333
50 0.11708496 0.96666664
0 0.00578535 1.0
50 0.0022929711 1.0
0 0.0010867945 1.0
50 0.0007732052 1.0


In [17]:
batch_iter = gen_batch(data, batch_size=batch_size)
next_batch = batch_iter.__next__()

In [18]:
next_batch[0][0]

array([0, 1, 6, 0, 6, 9, 5, 4, 2, 4, 5, 9, 4, 4, 3])

In [19]:
logger.setLevel(5)

In [20]:
sess.run(model(inp_embs(next_batch[0]), out_embs(next_batch[1][:,:-1])))

FailedPreconditionError: Attempting to use uninitialized value LayerNorm_8/beta
	 [[node LayerNorm_8/beta/read (defined at /home/arsen/anaconda3/envs/ng_fd/lib/python3.6/site-packages/tensorflow/contrib/framework/python/ops/variables.py:277)  = Identity[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](LayerNorm_8/beta)]]

Caused by op 'LayerNorm_8/beta/read', defined at:
  File "/home/arsen/anaconda3/envs/ng_fd/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/arsen/anaconda3/envs/ng_fd/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/arsen/anaconda3/envs/ng_fd/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/arsen/anaconda3/envs/ng_fd/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/arsen/anaconda3/envs/ng_fd/lib/python3.6/site-packages/ipyke

In [ ]:
def predict(enc_inp):
    for i in range(len(enc_inp))
    sess.run(out, feed_dict={enc_inp_seq: next_batch[0], dec_inp_seq:next_batch[1][:,:-1], out_lbls: get_labels(next_batch[1][:,1:])})